In [187]:
import requests as rq
import re
from bs4 import BeautifulSoup as bs
import numpy as np
import math
from random import randint
import json
import pickle
import time
import tqdm
import re
import datetime
from tqdm import tqdm
from getpass import getpass
import xmltodict, json
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)
pd.set_option('precision', 2)

pd.options.display.float_format = '{:20,.2f}'.format
np.set_printoptions(suppress=False)
from pymongo import MongoClient

In [16]:
def str2num(s):
    if s is None:
        return None
    try:
        return float(s)
    except ValueError:
        item = str(s).strip().replace(' ', '').replace(',', '.')
        res = re.search('[-+]?\d*\.?\d+', item)
        if res:
            return float(res.group(0))
        else:
            return None

def str2date(date):
    d = str(date)
    return pd.to_datetime(d[:10], format='%Y-%m-%d')

# Задание 2 урок

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [3]:
# Указываем интересующую должность, Headers и Paparams для запросов 
job = 'продавец'

In [4]:
headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36', 'accept': '*/*',
          'cookie': '_xsrf=de884125fde6477ed9c7161ed263ea53; _xsrf=de884125fde6477ed9c7161ed263ea53; hhrole=anonymous; region_clarified=NOT_SET; display=desktop; hhtoken=LPeQjTKZ_afPzaTypkHXCMgn0_jl; hhuid=lk36Wan1dMD2dl74cc1I5w--; GMT=3; _fbp=fb.1.1593340366222.421034606; iap.uid=2016ae8cd7d542b186d6e67a282f7d87; _ga=GA1.2.45243456.1593340367; _gid=GA1.2.1045293019.1593340367; __zzatgib-w-hh=MDA0dBA=Fz2+aQ==; regions=113; __ddg1=arLcdN8Pq8fTWee0ZdlV; _ym_d=1593345354; _ym_uid=159334535496613988; _ym_isad=2; __ddg2=sSwrGRG1bIhS7u2w; total_searches=47; gssc58=; _ym_visorc_156828=w; _gat_gtag_UA_11659974_2=1; cfidsgib-w-hh=yPYHkzwbbl0DO5VJDLh0f+Pjzkb47tVWctu3OAbo9LdPC+lj/9CGbxMAVInqh3ZK/WY/sYxuS1mnY++uDQ+YdSJjEIblX8A66VzewCULumH4nHpBVFhE6T8vOkE3ggxI+Z2Sy33+/mwRkPR7GBSIbk7x2AwQY0EZTjAisLY=; gsscgib-w-hh=NVIvBOyd9CDXzwpLBEvVN1HFf9QmBdIhGDZtZ5ZZ+ps4FZelgYFFK2WM0JAUKnxg2+p9DDGwrfxOPapKqLLkG00k7LEyCdZ8oas9CV4/8dnd8TmJAPbN9OeLhmTQg8vTiauJHU+e8b9PDX2SwgTqQFNmcBN2sahuEv2FuznPrHrndF/3mkhsqTJ4g94=; fgsscgib-w-hh=e0a6861354de74151cc564cc99569903e8742063; cfidsgib-w-hh=cpeq9lhFLTJephPmzT4JIGnNmwmBw8YzbF6rE7KzoX4e/EexNKNVn4RIe5ukHyCGwftXp9d6K1oHMsAoCMnIYJElCYMfks+/vkeg1M26QbfvpepfDBaDFFYKoS47ysDkcgtrpuZAzJYRbqKa8mlaMWF1l97+JMvTQ4jk3Ns='}
params = {'area':113, 'st':'searchVacancy', 'text':job,'fromSearch':True, 'from':'suggest_post', "items_on_page": 100}

*Теперь собираем максильно возможное количество страниц по каждому региону и добавляем ссылки на найденные страницы в список URLS*

In [5]:
areas = [i for i in range(0,5000)]

urls = []
for area in tqdm(areas):
    ur = f"https://hh.ru/search/vacancy?L_is_autosearch=false&area={area}&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text={params['text']}"
    req = rq.get(f"{ur}", headers=headers)
    soup = bs(req.text, 'html.parser')
    try:
        counts = int(str(soup.find_all("h1", attrs={'class':"bloko-header-1"})[0].text.strip()).split('вакансий')[0].replace('\xa0', ''))
    except:
        continue
    if counts > 0 and counts <= 100:
        urls.append([f"https://hh.ru/search/vacancy?L_is_autosearch=true&area={area}&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text={params['text']}&page=0"])
    if counts > 100:
        c = counts // 100 + 1
        res = c if c <=20 else 20
        urls.append([f"https://hh.ru/search/vacancy?L_is_autosearch=true&area={area}&currency_code=RUR&only_with_salary=true&items_on_page=100&clusters=true&enable_snippets=true&text={params['text']}&page={i}" for i in list(range(res))])                            

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [48:29<00:00,  2.55it/s]


In [6]:
with open('list_urls_hh_v3.pkl', 'wb') as f:
    pickle.dump(urls, f)

*Теперь собираем по списку URLS все id вакансий и создаем список ссылок LINKS уже на конкретные вакансии*

In [4]:
urls = pd.read_pickle('list_urls_hh_v3.pkl')

In [5]:
links = []
for url in tqdm(urls):
    for href in url:
#         print(f"{href}")        
        req = rq.get(f"{href}", headers=headers)
#         print(f"Статус: {req.status_code}")     
        soup = bs(req.text, 'html.parser')
        items1 = soup.find_all("script", attrs={"data-name":"HH/Vacancy/SendResponseAttempt"})
        for link in items1:
            data = dict(json.loads(link.get('data-params')))
            links.append(f"https://hh.ru/vacancy/{data['vacancyId']}?query={params['text']}")
links = list(set(links))

100%|████████████████████████████████████████████████████████████████████████████| 1696/1696 [5:33:39<00:00,  2.64it/s]


In [6]:
with open('final_links.pkl', 'wb') as f:
    pickle.dump(links, f)

***Собираем конечную информацию с отдельных конечных страничек вакансий*** 

In [170]:
links = pd.read_pickle('final_links.pkl')

In [171]:
data_f = []
count_ok = 0
for href in tqdm(links[:300]):
    time.sleep(randint(1,3))
    print(f"{href}")        
    req = rq.get(f"{href}", headers=headers)
    print(f"Статус: {req.status_code}")  
    if req.status_code > 400:
        continue
    soup = bs(req.text, 'html.parser')

    el = {}
    
    v1 = soup.find("script", attrs={'data-name':"HH/GoogleDfpService"})
    if v1:
        dd = v1.get('data-params')    
        salary_min = dict(json.loads(dd))['vac_salary_from']
        salary_max = dict(json.loads(dd))['vac_salary_to']
        lat = np.nan
        lon = np.nan
        employer = soup.find("span", attrs={'class':"bloko-section-header-2 bloko-section-header-2_lite"}).text
        city = soup.find("p", attrs={'data-qa':"vacancy-view-location"}).text
        
        ids1 = soup.find("script", attrs={'data-name':'HH/VacancyResponsePopup/VacancyResponsePopup'})
        if ids1:
            ids = ids1.get('data-params')
            id_point =  dict(json.loads(ids))['vacancyId']
        
        ids2 = soup.find("input", attrs={'name':'vacancyId'})
        if ids2:
            id_point = ids2.get('value')         
        
        currency = dict(json.loads(dd))['vac_salary_cur']
        
        date3 = soup.find("p", attrs={'class':"vacancy-creation-time"})
        if date3:
            date = date3.text.split('Вакансия опубликована')[1].split(' в ')[0].replace('\xa0', ' ').strip()  
            
        date2 = soup.find("span", attrs={'class':"vacancy-serp-item__publication-date"})
        if date2:
            date = date2.text.replace('\xa0', ' ')            
            
        date1 = soup.find("script", attrs={'type':"application/ld+json"})
        if date1:
            ff = date1.text
            date = dict(json.loads(ff))['datePosted']
            name = dict(json.loads(ff))['title']

        date0 = re.sub("[\r\t\n\b]","", soup.text)
        date0 = re.search(r"publicationDate\":\s\"(.*?)\"" , date0)
        if date0:
            date = date0.group(1)  
        

    v2 = soup.find("script", attrs={'data-name':"HH/Employer/VacancyCard/VacancyMap"})
    if v2:
        dd = v2.get('data-params')    
        salary_min = dict(json.loads(dd))['cianPlacemarks']['compensation']['from']
        salary_max = dict(json.loads(dd))['cianPlacemarks']['compensation']['to']
        lat = dict(json.loads(dd))['address']['mapData']['points']['marker']['lat']
        lon = dict(json.loads(dd))['address']['mapData']['points']['marker']['lng']
        city = dict(json.loads(dd))['address']['city']
        employer = soup.find("span", attrs={'class':"bloko-section-header-2 bloko-section-header-2_lite"}).text
        ids = soup.find("script", attrs={'data-name':"HH/Vacancy/SendResponseAttempt"}).get('data-params')
        id_point = dict(json.loads(ids))['vacancyId']
        currency = dict(json.loads(dd))['cianPlacemarks']['compensation']['currencyCode']
        
        if date:
            pass
        else:
            date = soup.find("span", attrs={'class':"vacancy-serp-item__publication-date"}).text.replace('\xa0', ' ')  
            
        date0 = re.sub("[\r\t\n\b]","", soup.text)
        date0 = re.search(r"publicationDate\":\s\"(.*?)\"" , date0)
        if date0:
            date = date0.group(1)            
        
    name = soup.find("h1", attrs={'data-qa':"vacancy-title"}).text
    
    el.update({
        'name' : name, 
        'link': href,
        'publish_date': date, 
        'salary_min': salary_min,
        'salary_max': salary_max,
        'lat': lat,
        'lon': lon,
        'city': city,
        'employer': employer,
        'id': id_point,
        'currency' : currency,
        'site': 'hh.ru'

    })
    data_f.append(el)
    count_ok += 1
df = pd.DataFrame(data_f)
hh = df.drop_duplicates('id')
hh.to_pickle('raw_data_v1.pkl')
print('Количество записей ', len(hh), 'Количество обработанных адресов', count_ok)




  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

https://hh.ru/vacancy/37649115?query=продавец
Статус: 200





  0%|▎                                                                                 | 1/300 [00:02<12:54,  2.59s/it]

https://hh.ru/vacancy/36745062?query=продавец
Статус: 200





  1%|▌                                                                                 | 2/300 [00:04<11:20,  2.28s/it]

...

https://hh.ru/vacancy/37740655?query=продавец
Статус: 200





100%|███████████████████████████████████████████████████████████████████████████████▋| 299/300 [12:09<00:03,  3.06s/it]

https://hh.ru/vacancy/37395343?query=продавец
Статус: 200





100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [12:10<00:00,  2.58s/it]

Количество записей  294 Количество обработанных адресов 299


# Задание 3 урок

- Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)
- Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)
- Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

***Готовим данные к загрузке в монгу***

In [172]:
hh = pd.read_pickle('raw_data_v1.pkl')

In [173]:
hh.shape

(294, 12)

***Небольшая обработка данных***

In [182]:
hh['publish_date'] = hh['publish_date'].apply(lambda x: str2date(x))

hh['salary_min'] = hh['salary_min'].apply(lambda x: str2num(x))
hh['salary_max'] = hh['salary_max'].apply(lambda x: str2num(x))
hh['lon'] = hh['lon'].apply(lambda x: str2num(x))
hh['lat'] = hh['lon'].apply(lambda x: str2num(x))
hh['id'] = hh['id'].apply(lambda x: str2num(x))

hh['salary_max'] = hh['salary_max'].apply(lambda x: np.nan if x == 0 else x)

hh['lat'] = hh['lat'].apply(lambda x: np.nan if x == 0 else x)
hh['lon'] = hh['lon'].apply(lambda x: np.nan if x == 0 else x)

hh['name'] = hh['name'].apply(lambda x: x.replace('\xa0', ' '))
hh['employer'] = hh['employer'].apply(lambda x: x.replace('\xa0', ' '))

In [183]:
hh.dtypes

name                    object
link                    object
publish_date    datetime64[ns]
salary_min             float64
salary_max             float64
lat                    float64
lon                    float64
city                    object
employer                object
id                     float64
currency                object
site                    object
dtype: object

In [184]:
# функция, которая будет добавлять в вашу базу данных только новые вакансии с сайта
def data_to_db(df):    
    client = MongoClient('localhost', 27017)
    hh_db = client['hh']
    vac = hh_db.vacancy
    have = list(pd.DataFrame(vac.find( {} ))['id'])
    rows = []
    vacs = {}
    for i, row in df[~(df['id'].isin(have))].iterrows():
        vacs = {
            'name' : row['name'],
            'link' : row['link'],
            'publish_date' : row['publish_date'],
            'salary_min' : row['salary_min'],
            'salary_max' : row['salary_max'],
            'lat' : row['lat'] if row['lat'] > 0 else None,
            'lon' : row['lon'] if row['lon'] > 0 else None,
            'city' : row['city'],
            'employer' : row['employer'],
            'id' : row['id'],
            'currency' : row['currency'],
            'site' : row['site']
        }
        rows.append(vacs)
    if len(rows) > 0:
        vac.insert_many(rows)
        print('Данные загружены')
        
    else:
        print('Нет новых записей')

In [185]:
# запускаем
data_to_db(hh)

Нет новых записей


In [186]:
# проверяем
pd.DataFrame(hh_db.vacancy.find( {} ))

,_id,name,link,publish_date,salary_min,salary_max,lat,lon,city,employer,id,currency,site
0,5efd83e56fa9af93f70e0836,"Продавец-кассир (Псков, улица Максима Горького, 19А)",https://hh.ru/vacancy/37649115?query=продавец,2020-06-24,"18,000.00","23,000.00",nan,nan,"Псков, улица Максима Горького, 19А",Пятёрочка. Магазины,"37,649,115.00",RUR,hh.ru
1,5efd83e56fa9af93f70e0837,"Продавец-кассир (Вологда, улица Панкратова, 80)",https://hh.ru/vacancy/36745062?query=продавец,2020-06-10,"18,000.00","23,000.00",39.83,39.83,Волода,Пятёрочка. Магазины,"36,745,062.00",RUR,hh.ru
2,5efd83e56fa9af93f70e0838,"Продавец-кассир (Москва, Флотская улица, 50)",https://hh.ru/vacancy/36911843?query=продавец,2020-06-29,"33,500.00","38,000.00",37.50,37.50,Москва,Пятёрочка. Магазины,"36,911,843.00",RUR,hh.ru
3,5efd83e56fa9af93f70e0839,Продавец-консультант автомобилей с пробегом Автопрайм,https://hh.ru/vacancy/37402498?query=продавец,2020-06-08,"100,000.00",nan,30.36,30.36,Санкт-Петербург,"РОЛЬФ, группа компаний","37,402,498.00",RUR,hh.ru
4,5efd83e56fa9af93f70e083a,Продавец-консультант (ТРЦ Гудзон),https://hh.ru/vacancy/37740408?query=продавец,2020-06-29,"38,000.00",nan,nan,nan,"Нагорная, Варшавская, Нахимовский проспект, Каширская, Москва, Каширское шоссе, 14",Henderson,"37,740,408.00",RUR,hh.ru
5,5efd83e56fa9af93f70e083b,"Продавец-кассир (Киров, Московская улица, 203)",https://hh.ru/vacancy/36468819?query=продавец,2020-06-08,"17,000.00","22,000.00",49.58,49.58,Киров,Пятёрочка. Магазины,"36,468,819.00",RUR,hh.ru
6,5efd83e56fa9af93f70e083c,Продавец-консультант,https://hh.ru/vacancy/37687064?query=продавец,2020-06-30,"41,000.00",nan,131.88,131.88,Владивосток,Связной,"11,445,422.00",RUR,hh.ru
7,5efd83e56fa9af93f70e083d,Продавец-консультант,https://hh.ru/vacancy/37455392?query=продавец,2020-06-11,"15,000.00","35,000.00",nan,nan,"Омск, улица 10 лет Октября, 219/1",ООО ФОРТУНА 55,"37,455,392.00",RUR,hh.ru
8,5efd83e56fa9af93f70e083e,Продавец-консультант (г. Печоры),https://hh.ru/vacancy/37222849?query=продавец,2020-06-30,"25,000.00",nan,nan,nan,Печоры,Связной,"37,222,849.00",RUR,hh.ru
9,5efd83e56fa9af93f70e083f,Продавец-консультант в магазин одежды (ул.Бориса Богаткова),https://hh.ru/vacancy/37501829?query=продавец,2020-06-16,"27,000.00",nan,nan,nan,"Новосибирск, улица Бориса Богаткова, 208/1",Торговая сеть OHARA,"37,501,829.00",RUR,hh.ru


In [181]:
# функция, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)
level = 22000
pd.DataFrame(hh_db.vacancy.find( {"$and": [ {"salary_min":{"$gt": level}}, {"salary_max":{"$gt": level}}]} ))

,_id,name,link,publish_date,salary_min,salary_max,lat,lon,city,employer,id,currency,site
0,5efd83e56fa9af93f70e0838,"Продавец-кассир (Москва, Флотская улица, 50)",https://hh.ru/vacancy/36911843?query=продавец,2020-06-29,"33,500.00","38,000.00",37.50,37.50,Москва,Пятёрочка. Магазины,"36,911,843.00",RUR,hh.ru
1,5efd83e56fa9af93f70e0843,Продавец-Управляющий шоу-румом одежды,https://hh.ru/vacancy/37601249?query=продавец,2020-06-22,"30,000.00","60,000.00",nan,nan,"Тюмень, улица Герцена, 64",ИП Липихин Андрей Николаевич,"37,601,249.00",RUR,hh.ru
2,5efd83e56fa9af93f70e0849,Продавец в бутик одежды Trussardi (Аутлет Новая Рига),https://hh.ru/vacancy/37583639?query=продавец,2020-07-01,"50,000.00","85,000.00",nan,nan,деревня Лапшинка,Т.Р.С. Дистрибьюшн,"37,583,639.00",RUR,hh.ru
3,5efd83e56fa9af93f70e084d,"Продавец-кассир (Нефтеюганск, с12)",https://hh.ru/vacancy/36469098?query=продавец,2020-06-24,"30,000.00","36,000.00",72.70,72.70,Нефтеюганск,Пятёрочка. Магазины,"36,469,098.00",RUR,hh.ru
4,5efd83e56fa9af93f70e0850,"Продавец-дизайнер в мебельный салон ""Инфинити"" (ТЦ Гранд)",https://hh.ru/vacancy/37668466?query=продавец,2020-06-30,"50,000.00","100,000.00",nan,nan,"Планерная, Химки, улица Бутаково, 4",INFINITI-Мебель,"37,668,466.00",RUR,hh.ru
5,5efd83e56fa9af93f70e0853,Продавец-консультант (м. новокосино),https://hh.ru/vacancy/37574874?query=продавец,2020-07-01,"48,000.00","80,000.00",nan,nan,"Москва, Новокосино",Билайн,"37,574,874.00",RUR,hh.ru
6,5efd83e56fa9af93f70e0854,"Продавец-кассир (ТЦ ""Республика"")",https://hh.ru/vacancy/37325355?query=продавец,2020-06-26,"25,500.00","25,500.00",nan,nan,Нижний Новгород,ПАО Детский Мир,"37,325,355.00",RUR,hh.ru
7,5efd83e56fa9af93f70e0856,"Продавец-кассир на АЗС (м. Академическая, Лесная, Финляндский вокзал, Ладожская)",https://hh.ru/vacancy/30386110?query=продавец,2020-06-29,"27,000.00","30,000.00",nan,nan,Санкт-Петербург,Стопэкспресс,"30,386,110.00",RUR,hh.ru
8,5efd83e56fa9af93f70e085a,"Старший продавец магазина ""Магнит Косметик"" (ЗЖМ)",https://hh.ru/vacancy/37640042?query=продавец,2020-06-23,"27,300.00","29,800.00",nan,nan,Ростов-на-Дону,"МАГНИТ, Розничная сеть","37,640,042.00",RUR,hh.ru
9,5efd83e56fa9af93f70e085b,"Продавец-консультант (Куйбышевский р-н, свет, электрика)",https://hh.ru/vacancy/37486090?query=продавец,2020-06-17,"23,000.00","25,000.00",nan,nan,"Новокузнецк, улица Орджоникидзе, 19",ООО Управляющая компания Техносистема,"37,486,090.00",RUR,hh.ru
